In [ ]:
import panel as pn
from bokeh.plotting import figure
import holoviews as hv
from holoviews.streams import Params
hv.extension('bokeh')
pn.extension()

In [ ]:
def make_volume():
    """This function makes a 3D array of data, a brain scan"""
    from skimage import io
    vol = io.imread(
#         "https://s3.amazonaws.com/assets.datacamp.com/blog_assets/" # Since the file has been downloaded
        "attention-mri.tif")
    return vol


TOOLTIPS = [
    ("x", "$x"),
    ("y", "$y"),
    ("value", "@image"),
]
shape = (157, 189, 68)
volume = make_volume()
inverted = False


In [ ]:
def make():
    """Here we make a figure widget with a 2D image and put it in the layout"""
    global inverted
    p1 = figure(width=400, height=400, tools='hover,wheel_zoom',
                tooltips=TOOLTIPS,
                x_range=[0, 157], y_range=[0, 189])
    if inverted:
        p1.image(image=[volume[:, ::-1, 30]], x=[0], y=[0], dw=[157], dh=[189])
    else:
        p1.image(image=[volume[:, :, 30]], x=[0], y=[0], dw=[157], dh=[189])
    widgets[1] = p1
    inverted = not inverted

In [ ]:
button = pn.widgets.Button(name='Invert')
button.on_click(lambda *_: make())
widgets = pn.Column(button, button)
make()
widgets


In [ ]:
volume.shape

### Simple Image

In [ ]:
hv.Image(volume[:,:,0])

In [ ]:
def get_brain_image(axes = 'XY',index = 0):
    if axes == 'XY':
        data = volume[:,:,index] 
    elif axes == 'XZ':
        data = volume[:,index,:]
    elif axes == 'YZ': 
        data = volume[index,:,:]
    
    image = hv.Image(data)
    return image

In [ ]:
dmap = hv.DynamicMap(get_brain_image, kdims=['index'])
dmap.redim.range(index=(0,67))